## Model 1: Here we are taking just the statements from the dataset and send this to a transformer and get percentage score of how confident the model is the statement is true 

In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [4]:
# Using the statement column from the original dataset, reaeding the data 
df = pd.read_csv('original_features.csv')
print(df.columns.tolist())


['json', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true_ct', 'false_ct', 'half_true_ct', 'mostly_true_ct', 'pants_on_fire_ct', 'context']


In [5]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [6]:
#making the train and test endcodings from statement 
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(list(train_df['statement']), padding='max_length', truncation=True, max_length=128)
test_encodings = tokenizer(list(test_df['statement']), padding='max_length', truncation=True, max_length=128)


In [7]:
# Custom PyTorch Dataset for tokenized text and labels

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    # online code used here
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, train_df['label'].tolist())
test_dataset = NewsDataset(test_encodings, test_df['label'].tolist())


In [8]:
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
training_args = TrainingArguments(
    output_dir='./results_section1',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    eval_strategy="epoch",          
    save_strategy="epoch",
    logging_dir='./logs_section1',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)


In [10]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)


In [12]:
# Training the model on 3 epochs
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.630000,0.657949,0.621250
2,0.586600,0.697057,0.597500
3,0.384600,0.910009,0.577500


TrainOutput(global_step=600, training_loss=0.5378586912155151, metrics={'train_runtime': 798.6469, 'train_samples_per_second': 12.02, 'train_steps_per_second': 0.751, 'total_flos': 317921756774400.0, 'train_loss': 0.5378586912155151, 'epoch': 3.0})

In [13]:
# saving the model

# Saving both the fine-tuned DistilBERT model and tokenizer 
save_directory = "section1/only_statement"

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


('section1/only_statement\\tokenizer_config.json',
 'section1/only_statement\\special_tokens_map.json',
 'section1/only_statement\\vocab.txt',
 'section1/only_statement\\added_tokens.json',
 'section1/only_statement\\tokenizer.json')

In [14]:
### Evaluate the model

predictions_output = trainer.predict(test_dataset)
#probabilities
probs = torch.nn.functional.softmax(torch.tensor(predictions_output.predictions), dim=1)[:, 1].numpy()
# Save to a dataframe
final_df = pd.DataFrame({
    'statement': test_df['statement'].values,
    'label': test_df['label'].values,
    'predicted_percentage_true': probs
})

final_df.to_csv('section1_predictions.csv', index=False)

print(final_df.head())


                                           statement  label  \
0  Obama says Iran is a 'tiny' country, 'doesn't ...  False   
1  Says Republican candidates in Oregon are advoc...  False   
2  Rob Cornilles has disputed for two years a new...  False   
3  I dont own a single stock or bond I have no sa...   True   
4  Regarding the Iraq War, I was the one that sai...  False   

   predicted_percentage_true  
0                   0.489328  
1                   0.457405  
2                   0.565089  
3                   0.524439  
4                   0.470493  


In [15]:
# Get predictions from Trainer output
predictions_output = trainer.predict(test_dataset)
# Extract predicted labels
y_pred = predictions_output.predictions.argmax(axis=1)
y_true = predictions_output.label_ids


In [16]:
#printing the metrics


from sklearn.metrics import accuracy_score, precision_score, recall_score

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

# Print metrics
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {2 * (precision * recall) / (precision + recall):.4f}")


Accuracy:  0.6212
Precision: 0.6335
Recall:    0.8589
F1 Score:  0.7292
